In [1]:
import os
import torchvision
from torchvision import models,transforms,datasets
import torch.nn as nn
import torch
import time
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
import imp
import utils; imp.reload(utils)
from utils import *

## Loading 2 dataset

In [3]:
upmc_data_dir = '/home/foodlovers/FoodVisor/data/images'

In [5]:
ezth_data_dir = '/mnt/food-101/images'

In [4]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(299),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(312),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

def shuffle_valtrain(x):
    if x == 'train':
        return True
    else :
        return False

In [5]:
upmc_dsets = {x: datasets.ImageFolder(os.path.join(upmc_data_dir, x), data_transforms[x])
         for x in ['train', 'test']}

In [6]:
upmc_dset_loaders = {x: torch.utils.data.DataLoader(upmc_dsets[x], batch_size=16,
                                               shuffle=shuffle_valtrain(x), num_workers=6)
                for x in ['train', 'test']}

In [6]:
ezth_dset = datasets.ImageFolder(ezth_data_dir)

In [32]:
ezth_dset.class_to_idx == upmc_dsets['train'].class_to_idx

True

In [8]:
ezth_dset.classes

['apple_pie',
 'baby_back_ribs',
 'baklava',
 'beef_carpaccio',
 'beef_tartare',
 'beet_salad',
 'beignets',
 'bibimbap',
 'bread_pudding',
 'breakfast_burrito',
 'bruschetta',
 'caesar_salad',
 'cannoli',
 'caprese_salad',
 'carrot_cake',
 'ceviche',
 'cheese_plate',
 'cheesecake',
 'chicken_curry',
 'chicken_quesadilla',
 'chicken_wings',
 'chocolate_cake',
 'chocolate_mousse',
 'churros',
 'clam_chowder',
 'club_sandwich',
 'crab_cakes',
 'creme_brulee',
 'croque_madame',
 'cup_cakes',
 'deviled_eggs',
 'donuts',
 'dumplings',
 'edamame',
 'eggs_benedict',
 'escargots',
 'falafel',
 'filet_mignon',
 'fish_and_chips',
 'foie_gras',
 'french_fries',
 'french_onion_soup',
 'french_toast',
 'fried_calamari',
 'fried_rice',
 'frozen_yogurt',
 'garlic_bread',
 'gnocchi',
 'greek_salad',
 'grilled_cheese_sandwich',
 'grilled_salmon',
 'guacamole',
 'gyoza',
 'hamburger',
 'hot_and_sour_soup',
 'hot_dog',
 'huevos_rancheros',
 'hummus',
 'ice_cream',
 'lasagna',
 'lobster_bisque',
 'lobster

In [7]:
ezth_dset.imgs[0]

('/mnt/food-101/images/apple_pie/1005649.jpg', 0)

In [10]:
classes_dict = {}

for i,item in enumerate(ezth_dset.imgs) :
    # print("{:d} / {:d}".format(i, len(ezth_dset.imgs)),end="\r")
    if item[1] not in classes_dict :
        classes_dict[item[1]] = [item[0]]
    else :
        classes_dict[item[1]].append(item[0])

7157 / 101000

26651 / 101000

31650 / 101000

36649 / 101000

50645 / 101000

...

58143 / 101000

68640 / 101000

77638 / 101000

90134 / 101000

93633 / 101000

In [23]:
from sklearn.model_selection import train_test_split
import os

ratio = 0.2
for i,food in enumerate(ezth_dset.classes) :
    print(food + " " + str(i))
    file_list = classes_dict[ezth_dset.class_to_idx[food]]
    file_list_train, file_list_test = train_test_split(file_list, test_size=ratio)
    os.system("sudo mkdir /mnt/ezth/images/train/"+food)
    os.system("sudo mkdir /mnt/ezth/images/test/"+food)
    for path in file_list_train :
        os.system("sudo scp "+path+" "+"/mnt/ezth/images/train/"+food)
    for path in file_list_test :
        os.system("sudo scp "+path+" "+"/mnt/ezth/images/test/"+food)

apple_pie 0
baby_back_ribs 1
baklava 2
beef_carpaccio 3
beef_tartare 4
beet_salad 5
beignets 6
bibimbap 7
bread_pudding 8
breakfast_burrito 9
bruschetta 10
caesar_salad 11
cannoli 12
caprese_salad 13
carrot_cake 14
ceviche 15
cheese_plate 16
cheesecake 17
chicken_curry 18
chicken_quesadilla 19
chicken_wings 20
chocolate_cake 21
chocolate_mousse 22
churros 23
clam_chowder 24
club_sandwich 25
crab_cakes 26
creme_brulee 27
croque_madame 28
cup_cakes 29
deviled_eggs 30
donuts 31
dumplings 32
edamame 33
eggs_benedict 34
escargots 35
falafel 36
filet_mignon 37
fish_and_chips 38
foie_gras 39
french_fries 40
french_onion_soup 41
french_toast 42
fried_calamari 43
fried_rice 44
frozen_yogurt 45
garlic_bread 46
gnocchi 47
greek_salad 48
grilled_cheese_sandwich 49
grilled_salmon 50
guacamole 51
gyoza 52
hamburger 53
hot_and_sour_soup 54
hot_dog 55
huevos_rancheros 56
hummus 57
ice_cream 58
lasagna 59
lobster_bisque 60
lobster_roll_sandwich 61
macaroni_and_cheese 62
macarons 63
miso_soup 64
mussels

In [7]:
ezth_dsets = {x: datasets.ImageFolder(os.path.join('/mnt/ezth/images/', x), data_transforms[x])
         for x in ['train', 'test']}

In [8]:
ezth_dset_loaders = {x: torch.utils.data.DataLoader(ezth_dsets[x], batch_size=16,
                                               shuffle=shuffle_valtrain(x), num_workers=6)
                for x in ['train', 'test']}

In [9]:
ezth_dset_sizes = {x: len(ezth_dsets[x]) for x in ['train', 'test']}
ezth_dset_sizes

{'train': 80800, 'test': 20200}

## Define model

In [10]:
model_inception = models.inception_v3(pretrained=True)

In [11]:
use_gpu = torch.cuda.is_available()
print('Using gpu: %s ' % use_gpu)

Using gpu: True 


In [12]:
model_inception.fc = nn.Linear(2048, 101)
model_inception.AuxLogits.fc = nn.Linear(768, 101)
if use_gpu:
    model_inception = model_inception.cuda()

In [31]:
model_inception.load_state_dict(torch.load('/home/foodlovers/FoodVisor/FoodVisor-TMM/trained_models/InceptionV3/model_current_1.pt'))

## Train model

In [13]:
criterion = nn.CrossEntropyLoss()
lr = 0.01
optimizer_incep = torch.optim.SGD(model_inception.parameters(),lr = lr, momentum=0.9)

In [19]:
path_save_1 = '/home/foodlovers/FoodVisor/FoodVisor-TMM/trained_models/InceptionV3/ezth_model_best.pt'
path_save_2 = '/home/foodlovers/FoodVisor/FoodVisor-TMM/trained_models/InceptionV3/ezth_model_current.pt'

def train_model(data,
                 epochs = 1,train = True, validate = False,
                shuffle = True) :
    
    if data == 'upmc' :
        train_loaders = upmc_dset_loaders['train']
        len_train = len(upmc_dsets['train'])
        test_loaders = upmc_dset_loaders['test']
        len_test = len(upmc_dsets['test'])
    elif data == 'ezth' :
        train_loaders = ezth_dset_loaders['train']
        len_train = len(ezth_dsets['train'])
        test_loaders = ezth_dset_loaders['test']
        len_test = len(ezth_dsets['test'])
    else :
        print("data must be 'upmc' or 'ezth' !")
        return
    
    if train == True :
        loss_history = []
        acc_history = []
        val_loss_history = []
        val_acc_history = []
        
    for epoch in range(epochs) :
        if train == True :
            #=========================TRAINING=================================#
            start_time_epoch = time.time()
            
            model_inception.train()
    
            print("Epoch:", epoch,"/",epochs,"===============================================")
        
            running_loss = 0.0
            running_corrects = 0.0


            for i, data in enumerate(train_loaders, 0):
                start_time = time.time()

                # get the inputs
                inputs, classes = data

                if use_gpu:
                    inputs , classes = inputs.cuda(), classes.cuda()

                # calulate outputs and losses
                outputs, aux_outputs = model_inception(inputs)
                loss = criterion(outputs,classes) + 0.4*criterion(aux_outputs,classes)         

                # autograd
                optimizer_incep.zero_grad()
                loss.backward()
                optimizer_incep.step()

                # statistics
                batch_loss = loss.data.item()
                _,preds = torch.max(outputs.data,1)
                batch_corrects = torch.sum(preds == classes.data)
                running_loss += batch_loss
                running_corrects += batch_corrects

                print('Batch {:d}/{:d} - Loss: {:.4f} Acc: {:.4f} - Time : {:.2f}s'.format(i+1,len(train_loaders),
                             batch_loss/len(classes), float(batch_corrects)/len(classes), time.time() - start_time), end="\r")

            epoch_loss = running_loss / len_train
            epoch_acc = running_corrects.data.item() / len_train
            
            loss_history.append(epoch_loss)
            acc_history.append(epoch_acc)
            
            print('Epoch {:d} completed in {:.2f} seconds ! Loss: {:.4f} Acc: {:.4f}'.format(
                     epoch, time.time() - start_time_epoch, epoch_loss, epoch_acc))
            
            torch.save(model_inception.state_dict(), path_save_2)
            
        if validate == True :
            #=========================VALIDATING=================================#
            
            model_inception.eval()
            
            val_loss = 0.0
            val_corrects = 0.0
            
            total = 0

            for i, data in enumerate(test_loaders, 0):
                start_time = time.time()
                
                # get the inputs
                inputs, classes = data

                if use_gpu:
                    inputs , classes = inputs.cuda(), classes.cuda()

                outputs = model_inception(inputs)

                loss = criterion(outputs,classes)        

                _,preds = torch.max(outputs.data,1)

                # statistics

                val_loss += loss.data.item()
                val_corrects += torch.sum(preds == classes.data)
                total += classes.size(0)
                
                print('Validating batch {:d}/{:d} - {:.2f}s ...'.format(i+1,len(test_loaders)
                                                                , time.time() - start_time), end="\r")

            val_epoch_loss = val_loss / len_test
            val_epoch_acc = val_corrects.data.item() / len_test

            print('Val Loss: {:.4f} Val Acc: {:.4f}'.format(
                             val_epoch_loss,val_epoch_acc))
            
            if train == False :
                return
            else :
                if len(val_acc_history) == 0 or val_epoch_acc >= max(val_acc_history) :
                    torch.save(model_inception.state_dict(), path_save_1)
                
                val_loss_history.append(val_epoch_loss)
                val_acc_history.append(val_epoch_acc)
    
    if train == False :
        return 'On fait rien!'
    elif validate == False :
        return loss_history, acc_history
    else :
        return loss_history, acc_history,val_loss_history,val_acc_history

In [33]:
train_model(data='ezth',
            train=False,validate=True,epochs = 10)

Val Loss: 0.1131 Val Acc: 0.61701s ...


In [16]:
history = train_model(data='ezth',
            train=True,validate=True,epochs = 10)

Epoch: 0 / 10 ===============================================
Epoch 0 completed in 3357.02 seconds ! Loss: 0.1990 Acc: 0.4446
Val Loss: 0.0902 Val Acc: 0.63481s ...
Epoch: 1 / 10 ===============================================
Epoch 1 completed in 5709.44 seconds ! Loss: 0.1642 Acc: 0.5364
Val Loss: 0.0784 Val Acc: 0.68961s ...
Epoch: 2 / 10 ===============================================
Epoch 2 completed in 3358.97 seconds ! Loss: 0.1461 Acc: 0.5827
Val Loss: 0.0718 Val Acc: 0.71201s ...
Epoch: 3 / 10 ===============================================
Epoch 3 completed in 3358.37 seconds ! Loss: 0.1346 Acc: 0.6156
Val Loss: 0.0652 Val Acc: 0.73041s ...
Epoch: 4 / 10 ===============================================
Epoch 4 completed in 3359.13 seconds ! Loss: 0.1252 Acc: 0.6403
Val Loss: 0.0601 Val Acc: 0.75471s ...
Epoch: 5 / 10 ===============================================
Epoch 5 completed in 3359.18 seconds ! Loss: 0.1175 Acc: 0.6600
Val Loss: 0.0587 Val Acc: 0.76261s ...
Epoch: 6 /

In [16]:
model_inception.load_state_dict(torch.load('/home/foodlovers/FoodVisor/FoodVisor-TMM/trained_models/InceptionV3/ezth_model_current.pt'))

In [17]:
criterion = nn.CrossEntropyLoss()
lr = 0.001
optimizer_incep = torch.optim.SGD(model_inception.parameters(),lr = lr, momentum=0.9)

In [18]:
history_2 = train_model(data='ezth',
            train=True,validate=True,epochs = 5)

Epoch: 0 / 5 ===============================================
Epoch 0 completed in 3357.74 seconds ! Loss: 0.0799 Acc: 0.7666


FileNotFoundError: [Errno 2] No such file or directory: '/home/foodlovers/FoodVisor/trained_models/InceptionV3/ezth_model_current.pt'

In [20]:
torch.save(model_inception.state_dict(), path_save_1)

In [21]:
train_model(data='ezth',
            train=False,validate=True,epochs = 10)

Val Loss: 0.0426 Val Acc: 0.83101s ...


In [22]:
history_2 = train_model(data='ezth',
            train=True,validate=True,epochs = 2)

Epoch: 0 / 2 ===============================================
Epoch 0 completed in 3369.12 seconds ! Loss: 0.0745 Acc: 0.7817
Val Loss: 0.0412 Val Acc: 0.83341s ...
Epoch: 1 / 2 ===============================================
Epoch 1 completed in 4235.02 seconds ! Loss: 0.0712 Acc: 0.7908
Val Loss: 0.0407 Val Acc: 0.83799s ...


In [23]:
train_model(data='upmc',
            train=False,validate=True,epochs = 10)

Val Loss: 0.1520 Val Acc: 0.58686s ...
